In [1]:
import pandas as pd
import numpy as np
import re, json  
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser


In [2]:
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())  
config['source']  #v v   .size().reset_index(name='counts')
ctx=context()
ctx.config=config

In [6]:

p=questionnaire_parser(ctx)
df=p.parse()
df=df.explode('Selections')
df

,ID,Question,Selections,DATATYPE,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,.GOV Domain Registration Systems,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Access to Data systems,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Agency support tool,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Business Intelligence Services,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Collaboration Tools (SharePoint),PICK,356
...,...,...,...,...,...
84,CQ32F,Is this Interconnection or data exchange a hig...,2. No,PICK,162
84,CQ32F,Is this Interconnection or data exchange a hig...,3. Unknown,PICK,162
85,CQ32G,"Is this interconnection Transmit, Receive or B...",1. Transmit,PICK,329
85,CQ32G,"Is this interconnection Transmit, Receive or B...",2. Receive,PICK,329


'C:\\Users\\timko\\Downloads\\custodian.xlsx'

In [6]:
df=pd.read_excel(config['source'], sheet_name=int(2), header=30, usecols='A,B,D').fillna(method='ffill').reset_index()
df=df.groupby(['ID','Question'], as_index=False).agg({'Selections': list, 'index': 'min'})
df=df.sort_values('index').reset_index(drop=True).drop('index', axis=1)
df['DATATYPE']=df['Selections'].apply(lambda x: 'TXT' if len(x) < 2 else 'PICK') 
pr = picklist_recommender(config['connstr']) 
df['PLT']=df['Selections'].apply(lambda x: pr.recommend(x)['PK_PicklistType'] ) 
pr.to_cache()
df  


,ID,Question,Selections,DATATYPE,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,"[.GOV Domain Registration Systems, Access to D...",PICK,356
1,CQ2,What services does the HVA system provide? Ple...,"[Cyber Security Support, Financial Information...",PICK,0
2,CQ3,To whom does the HVA provide services to? Plea...,"[Federal Agencies employees, Emergency respond...",PICK,355
3,CQ3,What is the approximate time in years since de...,"[< 5 years, > 6 – 10 years, > 10 – 15 years, ...",PICK,325
4,CQ4,Is this system identified as a Legacy* system ...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
...,...,...,...,...,...
81,CQ32C,Please indicate the organization type for the ...,"[1. Private/Commercial, 2. NGO, 3. FFRDC, 4. S...",PICK,0
82,CQ32D,Does the D/A for this HVA have an ISA/MOU with...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
83,CQ32E,Is this interconnection/data exchange mission ...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
84,CQ32F,Is this Interconnection or data exchange a hig...,"[1. Yes, 2. No, 3. Unknown]",PICK,162


In [151]:
df.explode('Selections')

,ID,Question,Selections,DATATYPE,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,.GOV Domain Registration Systems,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Access to Data systems,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Agency support tool,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Business Intelligence Services,PICK,356
0,CQ1,What is the purpose of the HVA? Please select ...,Collaboration Tools (SharePoint),PICK,356
...,...,...,...,...,...
84,CQ32F,Is this Interconnection or data exchange a hig...,2. No,PICK,162
84,CQ32F,Is this Interconnection or data exchange a hig...,3. Unknown,PICK,162
85,CQ32G,"Is this interconnection Transmit, Receive or B...",1. Transmit,PICK,329
85,CQ32G,"Is this interconnection Transmit, Receive or B...",2. Receive,PICK,329


In [150]:
df

,ID,Question,Selections,DATATYPE,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,"[.GOV Domain Registration Systems, Access to D...",PICK,356
1,CQ2,What services does the HVA system provide? Ple...,"[Cyber Security Support, Financial Information...",PICK,0
2,CQ3,To whom does the HVA provide services to? Plea...,"[Federal Agencies employees, Emergency respond...",PICK,355
3,CQ3,What is the approximate time in years since de...,"[< 5 years, > 6 – 10 years, > 10 – 15 years, ...",PICK,325
4,CQ4,Is this system identified as a Legacy* system ...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
...,...,...,...,...,...
81,CQ32C,Please indicate the organization type for the ...,"[1. Private/Commercial, 2. NGO, 3. FFRDC, 4. S...",PICK,0
82,CQ32D,Does the D/A for this HVA have an ISA/MOU with...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
83,CQ32E,Is this interconnection/data exchange mission ...,"[1. Yes, 2. No, 3. Unknown]",PICK,162
84,CQ32F,Is this Interconnection or data exchange a hig...,"[1. Yes, 2. No, 3. Unknown]",PICK,162


In [145]:
df['dtype']=df['Selections'].apply(lambda x: 'TXT' if len(x.split('\n')) < 2 else 'PICK') 
pr = picklist_recommender(config['connstr']) 
df['PLT']=df['Selections'].apply(lambda x: pr.recommend(x.split('\n'))['PK_PicklistType'] ) 
pr.to_cache()
df.head()

,ID,Question,Selections,dtype,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,.GOV Domain Registration Systems\nAccess to Da...,PICK,356
1,CQ2,What services does the HVA system provide? Ple...,Cyber Security Support\nFinancial Information ...,PICK,0
2,CQ3,To whom does the HVA provide services to? Plea...,Federal Agencies employees\nEmergency responde...,PICK,355
3,CQ3,What is the approximate time in years since de...,< 5 years\n> 6 – 10 years\n > 10 – 15 years\n ...,PICK,325
4,CQ4,Is this system identified as a Legacy* system ...,1. Yes\n2. No\n3. Unknown,PICK,162


In [144]:
df[0:35]

,ID,Question,Selections,dtype,PLT
0,CQ1,What is the purpose of the HVA? Please select ...,.GOV Domain Registration Systems\nAccess to Da...,PICK,356
1,CQ2,What services does the HVA system provide? Ple...,Cyber Security Support\nFinancial Information ...,PICK,0
2,CQ3,To whom does the HVA provide services to? Plea...,Federal Agencies employees\nEmergency responde...,PICK,355
3,CQ3,What is the approximate time in years since de...,< 5 years\n> 6 – 10 years\n > 10 – 15 years\n ...,PICK,325
4,CQ4,Is this system identified as a Legacy* system ...,1. Yes\n2. No\n3. Unknown,PICK,162
5,CQ4a,Is this system identified as a Legacy* system ...,"Legacy codebase (COBOL, MUMPS)\nLack of or poo...",PICK,326
6,CQ4b,Is there an approved Modernization or Technolo...,1. Yes\n2. No\n3. Unknown,PICK,162
7,CQ4c,Has the Modernization or Technology Plan been ...,1. Yes\n2. No\n3. Unknown,PICK,162
8,CQ5,Does this HVA have a decommissioning date with...,1. Yes\n2. No\n3. Unknown,PICK,162
9,CQ5a,"If so, what is the month and year of the plann...",DATE (MM/YYYY),TXT,0


In [90]:
re.search('.*free text.*',str('1. Free text').lower())

<re.Match object; span=(0, 12), match='1. free text'>

In [2]:
input="""
What is the approximate time in years since deployment of this HVA?
 
""".split('\n')

pr = picklist_recommender(csconnstr)
r=pr.recommend(input, threshhold=.8, usecosine_sim=False)
r





{'PK_PicklistType': 0,
 'UsageField': 0,
 'DisplayValue': 'WHAT IS THE APPROXIMATE TIME IN YEARS SINCE DEPLOYMENT OF THIS HVA',
 'PK_Picklist': 3419,
 'MAX_PK_Picklist': 10172,
 'MAX_PK_PicklistType': 387,
 'agg_simscore': 0.4166666666666667,
 'sequence_match_score': 0.4166666666666667}

In [16]:
df = sql_todf("""
SELECT PK_Picklist, PK_PicklistType, UsageField, DisplayValue from vwPicklists
""", csconnstr)  
df=df.loc[len(df)-1, ['PK_Picklist','PK_PicklistType']]
df

PK_Picklist        10172
PK_PicklistType      387
Name: 3069, dtype: object

In [4]:
def tokenize(s): 
    s = [v for v in s]  
    s = re.sub('[^A-Za-z0-9\s]','', ' '.join(s)).split() 
    return s

In [85]:
def normalize(s):  
    s = re.sub('[^A-Za-z0-9\s]',' ',s).upper().strip()
    s = re.sub('\s{1,5}',' ',s) 
    return s
normalize('   aASDF    ASDF   QWER QWER    QWERQWER')

'AASDF ASDF QWER QWER QWERQWER'

In [47]:
def jaccard_similarity(x,y): 
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [145]:

for i,r in df[167:168].iterrows():
    print(r['PK_PicklistType']) 
    x=normalize(r['DisplayValue'])
    print(x) 


294
NOT APPLICABLE ACQUIA ADOBE AT T AKAMAI AMAZON BLACKBERRY BOX CGI CISCO SYSTEMS EVERBRIDGE GOOGLE GRANICUS IBM LUMEN TECHNOLOGIES MICROSOFT MICROPACT ORACLE SAP SERVICENOW SPRINT VERIZON OTHER SALESFORCE


NameError: name 'picklist_recommender' is not defined

In [187]:
def recommend_picklist(input_list, threshhold=.8, usecosine_sim=False):
    df = sql_todf("SELECT PK_PicklistType, UsageField, DisplayValue from vwPicklists  ", csconnstr)  
    df = df.groupby(['PK_PicklistType','UsageField'], as_index = False).agg({'DisplayValue': ' '.join})       
    input=normalize(''.join(input_list))
    for i,r in df.iterrows(): 
        x=normalize(r['DisplayValue'])
        sequence_match = SequenceMatcher(None, x,input).ratio() 
        df.at[i,'sequence_match']= sequence_match   
        df.at[i,'cosine_sim'] = 0
        if df.at[i,'sequence_match'] > threshhold: 
            return df.iloc[i].to_dict()    
        if usecosine_sim:        
            vectorizer = TfidfVectorizer()
            X = vectorizer.fit_transform([x,input])  
            cosine_sim = cosine_similarity(X, X)  
            df.at[i,'cosine_sim']= cosine_sim[0][1]  
            if df.at[i,'cosine_sim'] > threshhold:
                return df.iloc[i].to_dict()
    df=df.sort_values(by=['cosine_sim', 'sequence_match'], ascending=[False,False], inplace=False)
    d=df.iloc[0].to_dict()    
    return d

input="""
1. Security Operations Center (SOC) as a Service such as Asset Management
2. Awareness and Training
3. Detection Processes
4. Governance: Information Protection Processes and Procedures;
5. Protective Technology
6. Risk Assessment
7. Security Continuous Monitoring
8. Supply Chain Risk Management

""".split('\n') 
recommend_picklist(input)

{'PK_PicklistType': 101,
 'UsageField': 'SAOP2',
 'DisplayValue': 'Policy making Compliance Risk management activities',
 'sequence_match': 0.1276595744680851,
 'cosine_sim': 0.0}

In [48]:
l=df.to_dict('records')
for d in l[:5]:
    print(d)
x = normalize(df['DisplayValue'].values)
x

['LEGACY CODEBASE COBOL MUMPS',
 'LACK OF OR POOR DOCUMENTATION',
 'LEGACY INFRASTRUCTURE',
 'AD HOC INFRASTRUCTURE',
 'HARDWARE OR SOFTWARE WITH NO VENDOR MANUFACTURER SUPPORT',
 'OUTDATED OR INACCURATE INVENTORY',
 'TECHNOLOGY TYPE',
 'SYSTEM LAST MODERNIZED',
 'LIMITED FUNCTIONALITY',
 'OTHER']

In [49]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
other
""".split('\n')
y= normalize(y)
y


['LEGACY CODEBASE COBOL MUMPS',
 'LACK OF OR POOR DOCUMENTATION',
 'LEGACY INFRASTRUCTURE',
 'AD HOC INFRASTRUCTURE',
 'HARDWARE OR SOFTWARE WITH NO VENDOR MANUFACTURER SUPPORT',
 'OUTDATED OR INACCURATE INVENTORY',
 'LIMITED FUNCTIONALITY',
 'OTHER',
 '']

0.8421052631578947
0.7272727272727273


[[1.         0.84904082]
 [0.84904082 1.        ]]


In [ ]:
threshold = .1
for x in range(0,X.shape[0]):
  for y in range(x,X.shape[0]):
    if(x!=y):
      if(cosine_similarity(X[x],X[y])>threshold):
        print(df["ID"][x],":",corpus[x])
        print(df["ID"][y],":",corpus[y])
        print("Cosine similarity:",cosine_similarity(X[x],X[y]))
        print()

In [51]:
corpus = x 
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
df_idf = pd.DataFrame(vectorizer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

<10x30 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [63]:
df = pd.DataFrame(columns=["ID","DESCRIPTION"], data=np.matrix([[10,"Cancel ASN WMS Cancel ASN"],
                                                                [11,"MAXPREDO Validation is corect"],
                                                                [12,"Move to QC"],
                                                                [13,"Cancel ASN WMS Cancel ASN"],
                                                                [14,"MAXPREDO Validation is right"],
                                                                [15,"Verify files are sent every hours for this interface from Optima"],
                                                                [16,"MAXPREDO Validation are correct"],
                                                                [17,"Move to QC"],
                                                                [18,"Verify files are not sent"]
                                                                ]))

corpus = list(df["DESCRIPTION"].values)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

threshold = 0.4

for x in range(0,X.shape[0]):
  for y in range(x,X.shape[0]):
    if(x!=y):
      if(cosine_similarity(X[x],X[y])>threshold):
        print(df["ID"][x],":",corpus[x])
        print(df["ID"][y],":",corpus[y])
        print("Cosine similarity:",cosine_similarity(X[x],X[y]))
        print()

10 : Cancel ASN WMS Cancel ASN
13 : Cancel ASN WMS Cancel ASN
Cosine similarity: [[1.]]

11 : MAXPREDO Validation is corect
14 : MAXPREDO Validation is right
Cosine similarity: [[0.64183024]]

12 : Move to QC
17 : Move to QC
Cosine similarity: [[1.]]

15 : Verify files are sent every hours for this interface from Optima
18 : Verify files are not sent
Cosine similarity: [[0.44897995]]



In [28]:

def sql_todf(query,connstr):
    df=pd.DataFrame() 
    engine = create_engine(connstr) 
    conn = engine.connect() 
    #try: 
    df = pd.read_sql(query,con=conn) 
    #finally: 
    conn.close()
    return df

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'TECHNOLOGY',
 'TYPE',
 '',
 'SYSTEM',
 'LAST',
 'MODERNIZED',
 'LIMITED',
 'FUNCTIONALITY',
 'OTHER',
 '']

In [58]:
y="""Legacy codebase (COBOL MUMPS)
Lack of or poor documentation
Legacy infrastructure
Ad-hoc infrastructure 
Hardware or software with no vendor/manufacturer support
Outdated or inaccurate inventory
Limited Functionality
""".replace('\n',' ').upper().split(' ')
y

['LEGACY',
 'CODEBASE',
 '(COBOL',
 'MUMPS)',
 'LACK',
 'OF',
 'OR',
 'POOR',
 'DOCUMENTATION',
 'LEGACY',
 'INFRASTRUCTURE',
 'AD-HOC',
 'INFRASTRUCTURE',
 '',
 'HARDWARE',
 'OR',
 'SOFTWARE',
 'WITH',
 'NO',
 'VENDOR/MANUFACTURER',
 'SUPPORT',
 'OUTDATED',
 'OR',
 'INACCURATE',
 'INVENTORY',
 'LIMITED',
 'FUNCTIONALITY',
 '']

In [59]:
x=lst

jaccard_similarity(x,y)

0.7931034482758621